# Topic Modeling for COVID19
COVID 19 has been the biggest pandemic people have seen in the recent times. It almost feels like one of those apocalyptic movies in real life. People hiding in their homes trying to save themselves from the infection. Some brave souls trying to find a better destination to survive this pandemic. With so much happening around the world, I have one question. **What are people around the world thinking about COVID 19?**

In this notebook, we will try to answer the above question using Topic Modeling. Let's explore the various topics people are talking about Corona Virus Disease 2019(COVID-19) in Twitter.

In [1]:
# Import all required libraries
import os
import json
import requests
import re

# Twitter data collection library
import tweepy as tw

# Data processing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Loading Gensim and nltk libraries
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
#Other stop_words: gensim.parsing.preprocessing.STOPWORDS


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/deepakawari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Configure notebook display to show data from pandas dataframe more clearly.
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',100)
pd.set_option('display.max_colwidth',1000)

# Step 1. Gather the textual data for Topic Modeling
To begin with the topic modeling we need the textual data. The textual data for what people are talking about COVID 19 can be pulled from many places such as social media, news articles, web scraping etc. In this notebook, we'll download the data from Twitter. Tweepy is an amazing library to pull data from twitter using your Twitter Developer Account. 

## Extracting tweets from Twitter API:
To load the data from Twitter using Tweepy API, you'll have to create Developer account with Twitter. Then download the credentials to authenticate using Tweepy API. **Please do not share these credentials with anybody else.**
* Here is the link to [apply for twitter developer access](https://developer.twitter.com/en/apply-for-access)
* You can follow the below code to use Tweepy API to authenticate and load the data. Here is the [Tweepy Documentation for reference](http://docs.tweepy.org/en/latest/) 

In [4]:
'''
LoadFromTwitter - 
    If true, pull the latest set of tweets from Twitter using the Tweepy library.
    If false, load the data from the datafile '../data/tweets.csv' if it exists, 
    otherwise load the tweets from Twitter using the Tweepy library.
    Set the LoadFromTwitter to True if you want to override loading the tweets afresh from twitter.
'''
LoadFromTwitter = False

fileName = '../data/tweets.csv'
tweetsDF = None

# Load the data
if os.path.exists(fileName) and not LoadFromTwitter:
    tweetsDF = pd.read_csv(fileName)
else:
    from TwitterDevSecrets import getTwitterDevCreds
    consumer_key, consumer_secret, access_token, access_secret = getTwitterDevCreds()

    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    # Set the wait_on_rate_limit and wait_on_rate_limit_notify to True
    # wait_on_rate_limit – 
    #    Whether or not to automatically wait for rate limits to replenish
    # wait_on_rate_limit_notify – 
    #    Whether or not to print a notification when Tweepy is waiting 
    #    for rate limits to replenish
    api = tw.API(
        auth, 
        wait_on_rate_limit=True, 
        wait_on_rate_limit_notify=True)

    # Define the search term and the date_since date as variables
    search_words = "#covid OR #covid19 OR #COVID OR #COVID19 OR #ncov OR #corona OR #coronaviru"
    date_since = "2020-05-16"
    
    # Read the tweets
    tweets = tw.Cursor(api.search, 
                   q=search_words,
                   lang="en",
                   since=date_since)

    # extract the data in pandas dataframe
    # Other parameters: tweet.user.screen_name, retweet_counts, favorite_counts
    tweetsDF = pd.DataFrame()
    for tweet in tweets.items(1000):
        id = tweet.id
        text = tweet.text
        loc = tweet.user.location
        tweetsDF = tweetsDF.append({'Id':id, 'Text':text, 'Location':loc},ignore_index=True)
    
    tweetsDF['index'] = tweetsDF.index
    
    # Save the new set of tweets in the file.
    tweetsDF.to_csv(fileName,index=False)



Let's see how doest he textual data look like.

In [7]:
tweetsDF.Text[1]

'RT @Carol_D_Johnson: Thank you nurses for helping to keep us healthy  ❤ #COVID19 \n#StayHomeSaveLives \n#coronavirus https://t.co/HGv0HfuTgt'

# Step 2. Data preprocessing
As you can see from the above text, a tweet contains a lot of textual data which probably doesn't contain any useful informaiton for Topic Modeling. So, these tweets needs to be processed to extract only useful textual data for further analysis. We will perform the following data processing steps:

* Tweet Preprocessing:
> * Remove the leading **RT** - RT indicates that the user is re-posting someone else's tweet. We can remove this token.
> * Remove the references to other accounts. The other accounts are usually referenced with '@' symbol.
> * Remove urls mentioned in the tweets.

* Generic text preprocessing:
> * **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
> * Remove words that have fewer than 3 characters.
> * Remove all **stopwords**. [Stop words](https://en.wikipedia.org/wiki/Stop_words) usually do not contain any usual information. As such these words are generally removed from the text in the preprocessing stage. 
> * **Lemmatize** the words: words in third person are changed to first person and verbs in past and future tenses are changed into present.  
> Lemmatization, unlike Stemming, reduces the inflected words properly ensuring that the root word belongs to the language. In Lemmatization root word is called Lemma. A lemma (plural lemmas or lemmata) is the canonical form, dictionary form, or citation form of a set of words. 
>> **WordnetLemmatizer**: uses lookup table from nltk wordnet corpus to lookup the lemma to return a valid language lemma.
> * **Stem** the Words: words are reduced to their root form.  
> Stemming is the process of reducing inflection in words to their root forms such as 
mapping a group of words to the same stem even if the stem itself is not a valid word 
in the Language.
>> **PorterStemmer**: is known for simplicity and ease. The algorithm does not follow linguistics rather a set of rules that are applied in phases (step by step) to generate stems. This is the reason why PorterStemmer does not often generate stems that are actual English words.  
>> **SnowballStemmer**: One can generate their own set of rules for any language. Python nltk introduced SnowballStemmers that are used to create non-English Stemmers!  
>> **LancasterStemmer**: is simple, but heavy stemming due to iterations and over-stemming may occur. Over-stemming causes the stems to be not linguistic, or they may have no meaning.
    

In [8]:
# Perform data preprocessing for all tweets.
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def tweet_cleanup(text):
    # Remove the leading RT from the tweet
    text = text.replace('RT','')
    # Remove the references to the account names starting with '@'
    text = re.sub(r'(@[a-zA-Z]*)','',text)
    # Remove the urls in the tweet.
    text = re.sub(r'((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)','',text)
    
    return text
  
# Clean up the tweets and then Tokenize and lemmatize
def preprocess(text, stop_words=stop_words):
    result=[]
    text = tweet_cleanup(text)
    for token in gensim.utils.simple_preprocess(text) :
        if token not in stop_words and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result



In [9]:
# Test the preprocessing step on a sample tweet
tweet_num = 0
sampleTweet = tweetsDF[tweetsDF['index'] == tweet_num].Text.iloc[0]

print("Original tweet: ")
words = []
for word in sampleTweet.split():
    words.append(word)
print(words)
print("\n\nPreprocessed tweet: ")
print(preprocess(sampleTweet))

Original tweet: 
['RT', '@ALPublicHealth:', 'State', 'Health', 'Officer', 'Dr.', 'Scott', 'Harris', 'has', 'issued', 'a', 'stay', 'at', 'home', 'order', 'and', 'strict', 'quarantine', 'requirements.', 'Read', 'our', 'full…']


Preprocessed tweet: 
['state', 'health', 'offic', 'scott', 'harri', 'issu', 'stay', 'home', 'order', 'strict', 'quarantin', 'requir', 'read', 'full']


In [10]:
# Preprocess all tweets and generate a new processed tweet text dataset.
processed_tweets = tweetsDF['Text'].map(preprocess)
processed_tweets[:10]

0        [state, health, offic, scott, harri, issu, stay, home, order, strict, quarantin, requir, read, full]
1                                                      [thank, nurs, help, keep, healthi, covid, coronavirus]
2                                                                        [togetherapart, slow, spread, covid]
3                    [smoker, greater, risk, contract, coronavirus, elli, cannon, say, equal, risk, contract]
4                                      [video, model, scan, show, extent, covid, damag, lung, tissu, stayhom]
5                      [leader, hous, parti, caucus, arizona, andi, bigg, think, spread, covid, much, possib]
6                                                                                     [covid, test, administ]
7           [keep, think, master, public, health, write, doctor, dissert, global, effort, tackl, aid, pandem]
8    [ceylonblacktea, rich, theaflavin, help, increas, human, immun, covid, srilankatea, industri, successfu]
9         

# Step 3. Text representation
Computers don't understand natural language texts. Text is a mere sequence of letters for computers. While its still difficult for computer to understand what the sequence of letters mean, language is way more complicated than that. For an example, let us consider an idiom "Kicked the bucket". You know where I am going right? When I first heard that phrase as a kid I thought it meant someone was actually kicking a bucket. That's fun! But, when I realized that it meant someone died, it was no more fun! So, natural language is hard and computers don't understand it. 

Computers love numbers. At the core, computers perform their operations on numbers. So, it'd be good to represent the natural language text with numbers for computer algorithms to process easily. In the below section, we'll explore two different models for text representation namely Bag of words and TF-IDF.

# 3.1: Bag of words on the dataset
Create a dictionary of words present in the preprocessed_tweets dataset. Gensim offers a great api for the same. This dictionary assigns a numerical id to each word so that you can work on the number representations of the word. This makes the data processing very easy than working on strings. 

Then create a corpus of Bag of words where words are represented by their numerical ids along with the frequency of occurence of that word in the tweet for further processing.

In [11]:
dictionary = gensim.corpora.Dictionary(processed_tweets)

# Create Corpus: Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in processed_tweets]

In [12]:
# Check the id to word mapping from the dictionary created above
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 5:
        break

0 full
1 harri
2 health
3 home
4 issu
5 offic


Since the text corpus is very huge and sparse, we should try to minimize the amount of text being used for modeling. For this reason, let us remove very rare and very common words. Gensim dictionary object provides a good api to perform this operation.
- words appearing less than 15 times
- words appearing in more than 10% of all documents

Then convert it into bag of word corpus with very rare and very common wordsd filtered out.

In [13]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_tweets]

# Test the Bag of Words representation of the tweet --> (token_id, token_count)
bow_corpus[tweet_num]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

In [14]:
# Preview BOW for our sample preprocessed tweet
bow_tweet_0 = bow_corpus[tweet_num]

for i in range(len(bow_tweet_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_tweet_0[i][0], 
                                                     dictionary[bow_tweet_0[i][0]], 
                                                     bow_tweet_0[i][1]))

Word 0 ("health") appears 1 time.
Word 1 ("home") appears 1 time.
Word 2 ("order") appears 1 time.
Word 3 ("state") appears 1 time.
Word 4 ("stay") appears 1 time.


# 3.2: TF-IDF on the data set
TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. Summing the Tf-idf of all possible terms and documents recovers the mutual information between documents and term taking into account all the specificities of their joint distribution.

TF (Term Frequency) - number of times a term occurs in a document.  
IDF (Inverse Document Frequency) diminishes the weight of terms that occur very frequently in the document set and increases the weight of terms that occur rarely.

In [21]:
# Create tf-idf model object using models.TfidfModel
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

# Apply transformation to the entire corpus
tfidf_corpus = tfidf[bow_corpus]

# Test the tf-idf representation of the sample tweet. Each word is represented by (token_id, tf-idf score).
tfidf_corpus[tweet_num]

[(0, 0.40009295170061265),
 (1, 0.44472924895798494),
 (2, 0.45253501051552114),
 (3, 0.47433139975516847),
 (4, 0.4608289406979316)]

In [22]:
# Preview TF-IDF for our sample preprocessed tweet
tfidf_tweet_0 = tfidf_corpus[tweet_num]

for i in range(len(tfidf_tweet_0)):
    print("Word {} (\"{}\") TF-IDF score: {}.".format(tfidf_tweet_0[i][0], 
                                                     dictionary[tfidf_tweet_0[i][0]], 
                                                     tfidf_tweet_0[i][1]))

Word 0 ("health") TF-IDF score: 0.40009295170061265.
Word 1 ("home") TF-IDF score: 0.44472924895798494.
Word 2 ("order") TF-IDF score: 0.45253501051552114.
Word 3 ("state") TF-IDF score: 0.47433139975516847.
Word 4 ("stay") TF-IDF score: 0.4608289406979316.


# Step 4: Topic modeling using LDA
Topic modeling is a statistical model to discover the abstract topics in a collection of documents. Probabilistic Latent Semantic Analysis(PLSA) is one of the ealiest models for topic modeling. Latent Dirichlet Allocation(LDA) is the most common topic model algorithm in use today which is a generalization of PLSA. 

LDA introduces sparse Dirichlet prior distributions over document-topic and topic-word distributions. This algorithm tries to model the intuition that each document has different abstract topics and that each topic is generalized by a small number of words.

In this section we'll be building the topic models using LDA for both text representations developed above. 


## Step 4.1: Modeling using Bag of Words
The LDA algorithm requires a few inputs to build the clusters. The main parameter it requires in the number of clusters we want the model to cluster the tweets into. But how do we identify the number of topics? The best way to identify that is by visualizing the clusters itself. 

Start with a high number of topics like 10 or 20. Then map the clusters into a vector space and see if the clusters have clear boundaries. If the clusters overlap, reduce the number of clusters, build the model and visualize again. Repeat the process until you are satisfied with the segregation of the clusters.

### Step 4.1.1: Running LDA using bag of words

In [28]:
# Train the lda model using gensim.models.LdaMulticore on Bag of word corpus
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=3, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

In [29]:
# Explore the words occuring in that topic and its relative weight
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.035*"mask" + 0.033*"keep" + 0.028*"stay" + 0.025*"home" + 0.021*"say" + 0.021*"human" + 0.020*"health" + 0.020*"order" + 0.020*"public" + 0.019*"time"


Topic: 1 
Words: 0.070*"coronavirus" + 0.043*"natur" + 0.042*"work" + 0.040*"speak" + 0.039*"caus" + 0.037*"father" + 0.036*"racism" + 0.036*"trut" + 0.036*"murder" + 0.035*"poverti"


Topic: 2 
Words: 0.072*"case" + 0.051*"test" + 0.043*"peopl" + 0.041*"make" + 0.038*"like" + 0.036*"today" + 0.035*"health" + 0.031*"fight" + 0.030*"death" + 0.027*"pandem"




From the top words in each topic, we can identify the generic topic in that cluster. In the above clustering, the topics could be around  
Topic 0: Self quarantining     
Topic 1: Impact of COVID-19 on work, racism, and poverty.   
Topic 2: Testing and health concerns due to COVID 19.  

### 4.1.2. Visualization using pyLDAVis for LDA with BOW
In this section, we'll visualize the topics generated by the above LDA model using pyLDAvis library. pyLDAvis provides an amazing interactive visualization tool to see how different clusters are generated. It produces the intertopic distance map and shows top relevant terms for each topic amongst other features. 

As mentioned above, we'll visualize the intertopic distance map to see if there is a good segregation of clusters. If there is an overlap between multiple clusters, we'd reduce the number of topics and run the LDA model with reduced number of clusters and visualize again. Once we are satisfied with the cluster segregation in the intertopic distance map, we can start looking into the terms to see what each topic represents.

In [16]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary=lda_model.id2word)
vis

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.199617 -0.001079       1        1  38.657166
0     -0.101144 -0.120415       2        1  33.898483
2     -0.098473  0.121495       3        1  27.444345, topic_info=    Category       Freq         Term      Total  loglift  logprob
20   Default  69.000000        peopl  69.000000  30.0000  30.0000
14   Default  49.000000       pandem  49.000000  29.0000  29.0000
34   Default  47.000000        natur  47.000000  28.0000  28.0000
35   Default  39.000000      poverti  39.000000  27.0000  27.0000
36   Default  39.000000       racism  39.000000  26.0000  26.0000
38   Default  39.000000         trut  39.000000  25.0000  25.0000
4    Default  33.000000         stay  33.000000  24.0000  24.0000
22   Default  41.000000         mask  41.000000  23.0000  23.0000
33   Default  40.000000       murder  40.000000  22.0000  22.0000
32   Default  41.000000       father  41.000000  21.0000  21.0000
1    Default  37.000000         home  37.000000  20.0000  20.0000
31   Default  42.000000         caus  42.000000  19.0000  19.0000
37   Default  44.000000        speak  44.000000  18.0000  18.0000
95   Default  22.000000       presid  22.000000  17.0000  17.0000
2    Default  33.000000        order  33.000000  16.0000  16.0000
0    Default  53.000000       health  53.000000  15.0000  15.0000
39   Default  58.000000         work  58.000000  14.0000  14.0000
42   Default  36.000000        death  36.000000  13.0000  13.0000
48   Default  74.000000         case  74.000000  12.0000  12.0000
7    Default  38.000000         keep  38.000000  11.0000  11.0000
84   Default  17.000000          ask  17.000000  10.0000  10.0000
13   Default  18.000000       global  18.000000   9.0000   9.0000
74   Default  18.000000         amaz  18.000000   8.0000   8.0000
70   Default  21.000000       worker  21.000000   7.0000   7.0000
27   Default  37.000000         like  37.000000   6.0000   6.0000
15   Default  36.000000       public  36.000000   5.0000   5.0000
47   Default  16.000000       import  16.000000   4.0000   4.0000
73   Default  15.000000         look  15.000000   3.0000   3.0000
9    Default  26.000000       spread  26.000000   2.0000   2.0000
12   Default  22.000000       doctor  22.000000   1.0000   1.0000
35    Topic1  38.575493      poverti  39.489479   0.9270  -3.3282
36    Topic1  38.574135       racism  39.490932   0.9269  -3.3283
38    Topic1  38.519722         trut  39.491779   0.9255  -3.3297
33    Topic1  39.085686       murder  40.457489   0.9159  -3.3151
32    Topic1  39.588772       father  41.396862   0.9058  -3.3023
31    Topic1  40.191837         caus  42.375614   0.8975  -3.2872
34    Topic1  44.587814        natur  47.048008   0.8967  -3.1834
37    Topic1  41.058357        speak  44.321198   0.8740  -3.2659
59    Topic1  13.655355         cast  15.107058   0.8494  -4.3667
50    Topic1  14.317590       enough  16.028320   0.8376  -4.3194
58    Topic1  13.418523         basi  15.123391   0.8308  -4.3842
64    Topic1  13.345068     religion  15.127920   0.8250  -4.3897
17    Topic1  14.825219         rich  17.015081   0.8127  -4.2845
66    Topic1  16.035984        india  18.943308   0.7838  -4.2060
39    Topic1  48.898685         work  58.742153   0.7670  -3.0911
67    Topic1  24.114191         make  29.275362   0.7565  -3.7980
61    Topic1  14.676572       either  17.998766   0.7464  -4.2946
48    Topic1  57.902523         case  74.930939   0.6926  -2.9221
43    Topic1  19.020647       report  25.739502   0.6479  -4.0353
60    Topic1  13.826070    discrimin  18.968874   0.6342  -4.3543
100   Topic1  10.937501       releas  15.166129   0.6236  -4.5886
57    Topic1  12.826177      respons  18.963476   0.5594  -4.4294
68    Topic1  19.123701        virus  28.531460   0.5504  -4.0299
62    Topic1  20.293621      everyon  30.528130   0.5421  -3.9705
63    Topic1  14.079419         poor  22.072075   0.5008 

Based on the visualizations, it'd be best to create 3 clusters instead of 10.

## 4.2: Modeling using TF-IDF
TF-IDF intends to reflect on the importance of each word in the tweet amongst other tweets. Thus it tries to create a better model instead of using mere Term Frequency as in Bag of words model. However, for TF-IDF to work it needs to have a good size of text in each document. However, tweet is usually very small in size. Thus, most of the times each word ends up being mentioned only once. Thus, TF-IDF doesn't work better for short texts. However, let's train the model and evaluate the performance and see how does it perform.

### 4.2.1. Running LDA using TF-IDF

In [25]:
# Train lda model using corpus_tfidf
lda_model_tfidf = gensim.models.LdaMulticore(tfidf_corpus, 
                                             num_topics=3, 
                                             id2word = dictionary, 
                                             passes = 2, 
                                             workers=4)

In [26]:
# Explore the words occuring in that topic and its relative weight
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.036*"trump" + 0.033*"mask" + 0.032*"order" + 0.032*"death" + 0.028*"call" + 0.027*"state" + 0.025*"pandem" + 0.025*"human" + 0.024*"presid" + 0.024*"worker"


Topic: 1 Word: 0.057*"coronavirus" + 0.039*"case" + 0.036*"make" + 0.034*"test" + 0.034*"health" + 0.027*"help" + 0.024*"patient" + 0.024*"home" + 0.024*"spread" + 0.022*"keep"


Topic: 2 Word: 0.054*"peopl" + 0.040*"work" + 0.032*"like" + 0.030*"natur" + 0.029*"fight" + 0.026*"caus" + 0.026*"father" + 0.026*"murder" + 0.026*"speak" + 0.025*"poverti"




Below is my attempt at generalizing the topics from their corresponding relevant terms.  
Topic 0: President Trump's announcements around COVID19.   
Topic 1: Quarantining and stopping the spread of the disease.  
Topic 2: Impact of COVID on people

### 4.2.2. Visualization using pyLDAVis

In [27]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, tfidf_corpus, dictionary=lda_model_tfidf.id2word)
vis

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.024509 -0.146795       1        1  37.265041
2     -0.120994  0.093648       2        1  33.191292
0      0.145502  0.053147       3        1  29.543669, topic_info=    Category       Freq         Term      Total  loglift  logprob
42   Default  22.000000         work  22.000000  30.0000  30.0000
22   Default  34.000000        peopl  34.000000  29.0000  29.0000
106  Default  19.000000        trump  19.000000  28.0000  28.0000
5    Default  41.000000  coronavirus  41.000000  27.0000  27.0000
37   Default  16.000000        natur  16.000000  26.0000  26.0000
36   Default  13.000000       murder  13.000000  25.0000  25.0000
2    Default  17.000000        order  17.000000  24.0000  24.0000
91   Default  14.000000         call  14.000000  23.0000  23.0000
38   Default  13.000000      poverti  13.000000  22.0000  22.0000
39   Default  13.000000       racism  13.000000  21.0000  21.0000
72   Default  23.000000         make  23.000000  20.0000  20.0000
41   Default  13.000000         trut  13.000000  19.0000  19.0000
35   Default  14.000000       father  14.000000  18.0000  18.0000
29   Default  18.000000         like  18.000000  17.0000  17.0000
34   Default  14.000000         caus  14.000000  16.0000  16.0000
18   Default  13.000000        human  13.000000  15.0000  15.0000
13   Default  23.000000         test  23.000000  14.0000  14.0000
40   Default  15.000000        speak  15.000000  13.0000  13.0000
0    Default  22.000000       health  22.000000  12.0000  12.0000
46   Default  19.000000        death  19.000000  11.0000  11.0000
24   Default  22.000000         mask  22.000000  10.0000  10.0000
45   Default  10.000000         isol  10.000000   9.0000   9.0000
102  Default  14.000000       presid  14.000000   8.0000   8.0000
75   Default  13.000000       worker  13.000000   7.0000   7.0000
70   Default  20.000000        fight  20.000000   6.0000   6.0000
61   Default  12.000000         must  12.000000   5.0000   5.0000
43   Default  15.000000       govern  15.000000   4.0000   4.0000
92   Default  13.000000        break  13.000000   3.0000   3.0000
3    Default  17.000000        state  17.000000   2.0000   2.0000
53   Default  31.000000         case  31.000000   1.0000   1.0000
92    Topic1  11.722426        break  13.075983   0.8778  -3.8723
65    Topic1   6.691922    discrimin   7.609566   0.8586  -4.4329
63    Topic1   4.820384         basi   5.560649   0.8443  -4.7610
64    Topic1   4.782580         cast   5.565120   0.8356  -4.7688
69    Topic1   4.777819     religion   5.567107   0.8342  -4.7698
72    Topic1  20.466206         make  23.936625   0.8305  -3.3151
49    Topic1  13.768491      patient  16.390236   0.8128  -3.7114
19    Topic1   5.424157         rich   6.505162   0.8054  -4.6430
100   Topic1   8.611781      confirm  10.379275   0.8004  -4.1807
0     Topic1  19.023029       health  22.952988   0.7993  -3.3882
8     Topic1   9.728037         nurs  11.752337   0.7981  -4.0588
10    Topic1  13.314896       spread  16.325382   0.7833  -3.7449
13    Topic1  19.203789         test  23.546783   0.7832  -3.3787
66    Topic1   5.734484       either   7.231761   0.7551  -4.5873
67    Topic1   9.720010      everyon  12.281673   0.7532  -4.0596
26    Topic1   9.872032        april  12.580767   0.7447  -4.0441
5     Topic1  32.347218  coronavirus  41.729073   0.7324  -2.8573
95    Topic1   8.109492         mani  10.521080   0.7268  -4.2408
27    Topic1   9.109095     lockdown  12.277693   0.6886  -4.1246
1     Topic1  13.374413         home  18.487516   0.6634  -3.7405
7     Topic1  12.619587         keep  17.490751   0.6607  -3.7986
44    Topic1   7.506304         even  10.562677   0.6455  -4.3181
6     Topic1  15.177083         help  21.381548   0.6444  -3.6140
53    Topic1  21.761744         case  31.034859   0.6322  -3.2537
68    Topic1   6.003836         poor   8.579297   0.6302 

# Step 5. Model evaluation
Topic cluster is a statistical tool that imposes probability distribution over words. We can use different coherence measures to quantify the quality of the clusters and the probability distribution of the words. However, before we dwelve into the coherence measures, lets try to approach the evaluation with a more intuitive way i.e. human validation. 

## 5.1. Evaluation by human validation
In this section, we'll cluster a sample tweet and see how well does this clustering matches the overall topics generated above.

### 5.1.1: Human validation for Bag of words model
Classify a sample tweet into the topics and then evaluate if the general topic matches with the tweet better than other topics.

In [30]:
# Our test tweet is 
print('Our test tweet is: {}: {}'.format(tweet_num, [dictionary[word[0]] for word in bow_corpus[tweet_num]]))

for index, score in sorted(lda_model[bow_corpus[tweet_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {} : {}".format(score, index, lda_model.print_topic(index, 10))) 

Our test tweet is: 0: ['health', 'home', 'order', 'state', 'stay']

Score: 0.882530689239502	 
Topic: 0 : 0.035*"mask" + 0.033*"keep" + 0.028*"stay" + 0.025*"home" + 0.021*"say" + 0.021*"human" + 0.020*"health" + 0.020*"order" + 0.020*"public" + 0.019*"time"

Score: 0.060176584869623184	 
Topic: 2 : 0.072*"case" + 0.051*"test" + 0.043*"peopl" + 0.041*"make" + 0.038*"like" + 0.036*"today" + 0.035*"health" + 0.031*"fight" + 0.030*"death" + 0.027*"pandem"

Score: 0.05729272961616516	 
Topic: 1 : 0.070*"coronavirus" + 0.043*"natur" + 0.042*"work" + 0.040*"speak" + 0.039*"caus" + 0.037*"father" + 0.036*"racism" + 0.036*"trut" + 0.036*"murder" + 0.035*"poverti"


The sample tweet is classified to Topic 0 with 88% score. Topic 0 in Bag of word model was centered around self quarantining. The sample tweet matches with this topic. We could try evaluating more tweets manually. Seems like the BOW based LDA model worked well.    

### 5.1.2 Human validation for TF-IDF model

In [31]:
# Our test tweet is 
print('Our test tweet is: {}: {}'.format(tweet_num, [dictionary[word[0]] for word in tfidf_corpus[tweet_num]]))

for index, score in sorted(lda_model_tfidf[tfidf_corpus[tweet_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {} : {}".format(score, index, lda_model.print_topic(index, 10)))

Our test tweet is: 0: ['health', 'home', 'order', 'state', 'stay']

Score: 0.48965832591056824	 
Topic: 1 : 0.070*"coronavirus" + 0.043*"natur" + 0.042*"work" + 0.040*"speak" + 0.039*"caus" + 0.037*"father" + 0.036*"racism" + 0.036*"trut" + 0.036*"murder" + 0.035*"poverti"

Score: 0.38157591223716736	 
Topic: 0 : 0.035*"mask" + 0.033*"keep" + 0.028*"stay" + 0.025*"home" + 0.021*"say" + 0.021*"human" + 0.020*"health" + 0.020*"order" + 0.020*"public" + 0.019*"time"

Score: 0.1287657469511032	 
Topic: 2 : 0.072*"case" + 0.051*"test" + 0.043*"peopl" + 0.041*"make" + 0.038*"like" + 0.036*"today" + 0.035*"health" + 0.031*"fight" + 0.030*"death" + 0.027*"pandem"


As can be seen above, the sample tweet is split between topics 0 with 49% score and topic 1 with 38% score. As we saw in the section 4.2.1, Topic 0 was centered around President Trump's announcement around COVID 19 and Topic 1 around Quarantine and fight the spread of COVID-19. 

The sample tweet matches well with Topic 1. However, the confidence of this clustering is low compared to 88% confidence we saw for Bag of Words model. 

The TF-IDF modeling didn't have good confidence in the classification. This was expected as TF-IDF doesn't work good for short text documents. 